In [33]:
import geopandas as gpd
from scipy.spatial.distance import cdist
from geopy.distance import geodesic
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import shapely.wkt
from pyproj import Proj, transform
import numpy as np



In [34]:
# Load a shapefile
filepath = 'land_designations/GPL0.shp'  
data = gpd.read_file(filepath)

In [35]:
print(data.head())

   MAVAT_CODE           MAVAT_NAME STATUS ADDRESS    Shape_STAr  Shape_STLe  \
0         850  דרך ו/או טיפול נופי   None    None   8295.212296  916.653343   
1         830            דרך מוצעת   None    None  25455.303329  844.585314   
2         880         מסילה מאושרת   None    None  10053.510155  895.613171   
3         930       מפגש דרך-מסילה   None    None   2820.140495  298.743373   
4         830            דרך מוצעת   None    None   6017.871506  358.949246   

                                            geometry  
0  POLYGON ((218293.440 628459.297, 218290.925 62...  
1  POLYGON ((218463.073 628482.191, 218461.392 62...  
2  POLYGON ((218336.288 628338.636, 218314.935 62...  
3  POLYGON ((218252.954 628346.817, 218267.869 62...  
4  POLYGON ((218494.656 628594.806, 218496.861 62...  


In [36]:
# Print unique values in the column 'MAVAT_NAME'
unique_values = data['MAVAT_NAME'].unique()
print(unique_values)

['דרך ו/או טיפול נופי' 'דרך מוצעת' 'מסילה מאושרת' 'מפגש דרך-מסילה'
 "מגורים ד'" 'שטח ציבורי פתוח' 'מבנים ומוסדות ציבור' 'מגורים מסחר ותעסוקה'
 "מגורים ג'" 'שטח פרטי פתוח' 'דרך מאושרת' "מגורים ב'"
 'יעוד עפ"י תכנית מאושרת אחרת' 'מגורים מסחר ומבנים ומוסדות'
 'מבנים ומוסדות ציבור לחינוך' 'שביל' 'מגורים מסחר ותיירות' 'מגורים'
 "מגורים א'" 'יער' 'מגורים ומסחר' 'מבנים ומוסדות ציבור לחרום ו'
 'מסחר ומשרדים' 'דרך משולבת' 'שטח שהתוכנית אינה חלה עליו'
 'מלונאות (אכסון מלונאי)' 'מסילת ברזל מוצעת' 'מבנים מוסדות ציבור ותחבורה'
 "מגורים א' 3" "מגורים ב' 2" "îâåøéí â'" 'îâåøéí' 'îñçø'
 'îáðéí åîåñãåú öéáåø' 'ùèç öéáåøé ôúåç' 'שטחים פתוחים'
 'שטחים פתוחים ומבנים ומוסדו' 'דיור מיוחד' "מגורים ג' 1" "מגורים א' 2"
 'מסחר' 'מגורים ומבנים ומוסדות ציבו' 'מבנים ומוסדות ציבור לדת'
 "מגורים ב' 3" 'גן לאומי' 'תעשיה' 'מגורים ודיור מיוחד' 'עירוני מעורב'
 'תחנת תדלוק' 'חניון' 'שטח לתפעול מסילה' 'ספורט ונופש'
 "מגורים ג' 2 - יעוד מבוטל" 'גבול תכנית ארצית/מחוזית'
 'מסחר תעסוקה ומבנים ומוסדות' 'מסחר תעסוקה ותיירות' 'ש

In [37]:
filtered_data = data[data['MAVAT_NAME'] == 'שטח ציבורי פתוח']
print(filtered_data)

       MAVAT_CODE       MAVAT_NAME STATUS ADDRESS    Shape_STAr  Shape_STLe  \
6             670  שטח ציבורי פתוח   None    None   1708.090652  237.107600   
8             670  שטח ציבורי פתוח   None    None    207.585426  116.530002   
12            670  שטח ציבורי פתוח   None    None  15141.456818  930.543866   
16            670  שטח ציבורי פתוח   None    None    278.703682  100.268871   
23            670  שטח ציבורי פתוח   None    None   1943.602951  207.914627   
...           ...              ...    ...     ...           ...         ...   
21262         670  שטח ציבורי פתוח   None    None    966.542023  173.309903   
21272         670  שטח ציבורי פתוח   None    None    774.370644  122.029605   
21280         670  שטח ציבורי פתוח   None    None    312.917786   70.816985   
21307         670  שטח ציבורי פתוח   None    None     95.670876   56.481157   
21308         670  שטח ציבורי פתוח   None    None    361.145851   82.112171   

                                                geo

In [38]:
print(filtered_data['geometry'].area)

6         1708.090667
8          207.585435
12       15141.456811
16         278.703679
23        1943.602964
             ...     
21262      966.542017
21272      774.370653
21280      312.917806
21307       95.670882
21308      361.145843
Length: 1960, dtype: float64


In [39]:
# Function to convert a string "(x, y)" to a Point object
def str_to_point(s):
    x, y = map(float, s.strip("()").split(","))
    return Point(x, y)

In [40]:
buildings_df = pd.read_csv('מבנים גאו.csv', encoding='utf-8')
buildings_df['geometry'] = buildings_df['geo'].apply(str_to_point)
buildings_gdf = gpd.GeoDataFrame(buildings_df, geometry='geometry')
buildings_gdf

,Unnamed: 0,OBJECTID,סמל רחוב,שם רחוב,מספר בית,קוד שימוש,שימוש במבנה,מספר קומות,מספר כניסות,מספר דירות,שכבה,קוד רחוב 2,שם רחוב 2,Shape,שטח מעטפת במטר מרובע,היקף מעטפת במטרים,geo,geometry
0,0,36682972,1117,קורות העיתים,6.0,,,4,2,16,2200,0,,NaN,345.728714,107.004590,"(218666.4219, 632825.4844)",POINT (218666.422 632825.484)
1,1,36682973,988,הרב שרירא גאון,4.0,,,4,3,3,2200,0,,NaN,269.410110,91.708568,"(218666.4219, 632825.4844)",POINT (218666.422 632825.484)
2,2,36682974,1117,קורות העיתים,8.0,,,3,1,10,2200,0,,NaN,275.475449,87.499202,"(218647.0156, 632839.6719)",POINT (218647.016 632839.672)
3,3,36682975,1090,העילוי,12.0,,,4,1,11,2200,0,,NaN,192.787643,75.604930,"(218622.1571, 632908.762700001)",POINT (218622.157 632908.763)
4,4,36682976,1086,בן ציון,14.0,,,4,1,10,2200,0,,NaN,302.826126,72.239101,"(218585.9121, 632910.493799999)",POINT (218585.912 632910.494)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,36678641,4398,אל קד'איל,0.0,,,2,1,2,2200,0,,NaN,438.811554,127.739575,"(219115.9071, 628511.206)",POINT (219115.907 628511.206)
996,996,36678642,4398,אל קד'איל,0.0,,,2,1,2,2200,0,,NaN,231.474670,80.553346,"(219115.9071, 628511.206)",POINT (219115.907 628511.206)
997,997,36678643,4285,איחוד הכפר,0.0,,,2,2,3,2200,0,,NaN,355.761520,90.683741,"(219115.9071, 628511.206)",POINT (219115.907 628511.206)
998,998,36678644,4398,אל קד'איל,0.0,,,3,1,2,2200,0,,NaN,238.064072,76.818314,"(219115.9071, 628511.206)",POINT (219115.907 628511.206)


In [41]:
gardens_gdf = gpd.GeoDataFrame(filtered_data, geometry='geometry')
gardens_gdf

,MAVAT_CODE,MAVAT_NAME,STATUS,ADDRESS,Shape_STAr,Shape_STLe,geometry
6,670,שטח ציבורי פתוח,None,None,1708.090652,237.107600,"POLYGON ((216366.419 628862.605, 216355.647 62..."
8,670,שטח ציבורי פתוח,None,None,207.585426,116.530002,"POLYGON ((216214.054 628905.720, 216199.583 62..."
12,670,שטח ציבורי פתוח,None,None,15141.456818,930.543866,"POLYGON ((218574.604 628562.437, 218569.990 62..."
16,670,שטח ציבורי פתוח,None,None,278.703682,100.268871,"POLYGON ((218550.782 628584.476, 218544.050 62..."
23,670,שטח ציבורי פתוח,None,None,1943.602951,207.914627,"POLYGON ((218728.546 632549.604, 218729.397 63..."
...,...,...,...,...,...,...,...
21262,670,שטח ציבורי פתוח,None,None,966.542023,173.309903,"POLYGON ((219834.174 629857.900, 219836.638 62..."
21272,670,שטח ציבורי פתוח,None,None,774.370644,122.029605,"POLYGON ((218437.542 629012.265, 218427.200 62..."
21280,670,שטח ציבורי פתוח,None,None,312.917786,70.816985,"POLYGON ((221515.333 638007.209, 221512.658 63..."
21307,670,שטח ציבורי פתוח,None,None,95.670876,56.481157,"POLYGON ((216402.770 628859.500, 216401.658 62..."


In [42]:
gardens_gdf['geometry'] = gardens_gdf['geometry'].centroid
gardens_gdf

,MAVAT_CODE,MAVAT_NAME,STATUS,ADDRESS,Shape_STAr,Shape_STLe,geometry
6,670,שטח ציבורי פתוח,None,None,1708.090652,237.107600,POINT (216345.815 628835.669)
8,670,שטח ציבורי פתוח,None,None,207.585426,116.530002,POINT (216209.727 628899.593)
12,670,שטח ציבורי פתוח,None,None,15141.456818,930.543866,POINT (218461.704 628432.494)
16,670,שטח ציבורי פתוח,None,None,278.703682,100.268871,POINT (218553.404 628594.102)
23,670,שטח ציבורי פתוח,None,None,1943.602951,207.914627,POINT (218703.518 632519.830)
...,...,...,...,...,...,...,...
21262,670,שטח ציבורי פתוח,None,None,966.542023,173.309903,POINT (219849.011 629832.484)
21272,670,שטח ציבורי פתוח,None,None,774.370644,122.029605,POINT (218418.866 629008.051)
21280,670,שטח ציבורי פתוח,None,None,312.917786,70.816985,POINT (221504.515 638000.644)
21307,670,שטח ציבורי פתוח,None,None,95.670876,56.481157,POINT (216391.778 628863.369)


In [43]:
# Calculate the distance between each building and each garden
buildings_gdf['geometry'] = buildings_gdf['geometry'].apply(lambda x: x.centroid)

def calculate_distance(building, garden):
    # Assuming the units of the coordinates are meters, you can convert them to kilometers for distance calculation
    return building.distance(garden) / 1000

# Create a new column in gdf_buildings to store the array of gardens within 7 minutes walking distance
buildings_gdf['garden'] = np.nan

# Iterate over each building and find gardens within 7 minutes walking distance
for idx, building in buildings_gdf.iterrows():
    distance_to_gardens = gardens_gdf['geometry'].apply(lambda x: calculate_distance(building['geometry'], x))
    gardens_within_7_minutes = gardens_gdf[distance_to_gardens <= 0.7]  # Assuming 1 minute is approximately 100 meters
    buildings_gdf.at[idx, 'garden'] = gardens_within_7_minutes.to_numpy()

# Drop the geometry column from gdf_buildings as it's no longer needed
buildings_gdf.drop(columns=['geometry'], inplace=True)

ValueError: Must have equal len keys and value when setting with an ndarray

In [44]:
import pyproj

# Define the transformer from Israel TM to WGS84 (latitude and longitude)
transformer = pyproj.Transformer.from_proj(
    pyproj.Proj('epsg:2039'),  # Israel Transverse Mercator
    pyproj.Proj('epsg:4326'))   # WGS84 (latitude and longitude)

# Example coordinates in Israel format
israel_x, israel_y = 218666.4219, 632825.4844

# Convert the coordinates to latitude and longitude
longitude, latitude = transformer.transform(israel_x, israel_y)

print(f"Latitude: {latitude}, Longitude: {longitude}")


Latitude: 35.19610060773515, Longitude: 31.788134716783595
